In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import cv2
import os
import re
import pickle
import math
from pathlib import *
import multiprocessing
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from collections import *
from itertools import *
from functools import *
from sklearn.metrics import *
from scipy.stats import *
import pandas as pd
import seaborn as sns
import hashlib
from PIL import Image
from datastatics import *
from oct_Utils import *
import itertools

/home/octusr3/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
map_matrix='''[ [0  0  0  0  0  0  0  0  0  0]
                [0  0  0  46 44 43 42 0  0  0]
                [0  0  46 45 44 43 41 43 0  0]
                [0  45 46 45 45 45 42 42 41 0]
                [46 46 47 46 50 50 50 3  41 0]
                [9  9  9  6  8  6  7  3  16 0]
                [0  10 8  7  6  8  9  13 17 0]
                [0  0  8  8  10 10 12 15 0  0]
                [0  0  0  9  10 11 14 0  0  0]
                [0  0  0  0  0  0  0  0  0  0]]'''#为了凑54个点，给两个视野盲区的点都随便指向了一个2


def str_to_np_array(row, dtype=np.float):
    row = row.replace('[', '').replace(']', '')
    row = row.split(' ')
    row = filter(lambda x: x != '', row)
    row = map(lambda x: dtype(x), row)
    row = np.array(list(row))
    return row

def str_to_np_mat(s, dtype=np.float):
    s = s[1:-1]
    rows = s.split('\n')
    for index, row in enumerate(rows):
        rows[index] = str_to_np_array(row, dtype)
    rows = np.stack(rows, axis=0)
    return rows


def calculate_position(arr):
	#拼接数组函数
    array_num = list(itertools.chain.from_iterable(arr))
    array_num = np.array(array_num)
    key = np.unique(array_num)
    result = {}
    for k in key:
        index=np.argwhere(arr == k)#返回所有符合条件的索引值，更多的方法和类参数需要看
        v=[tuple(index[i]) for i in range(0,len(index))]
        result[k] = v
    result = {key:val for key, val in result.items() if  len(val)<9}

    valid_point=[]
    for key,value in result.items():
        valid_point+=value

    return result,valid_point

num_mat=str_to_np_mat(map_matrix)
countNums,valid_point=calculate_position(num_mat)#之前statics的时候，统计的数据是用的repeat的重复数据，这次要重新处理，尽量保证模块化，工程化
point_to_image_slice=reverse_dict(countNums)

In [3]:
data=pd.read_csv('data.csv')#
data['np_mat_num']=data['num'].apply(str_to_np_mat)
data['np_mat_num']=data['np_mat_num'].mask(data['eye']=='OD',data['np_mat_num'].apply(lambda x:np.flip(x,axis=1)))

In [4]:
type(data["np_mat_num"])

pandas.core.series.Series

In [23]:
data=pd.read_csv('data.csv')#
data['np_mat_num']=data['num'].apply(str_to_np_mat)
data['np_mat_num']=data['np_mat_num'].mask(data['eye']=='OD',data['np_mat_num'].apply(lambda x:np.flip(x,axis=1)))#all_num已经翻转过，所以数据是正常的,这里相当于
#把label直接翻转好了，后续根据input直接做就可以了
all_num=data['np_mat_num']

def split_np_matrix(all_num,data:pd.DataFrame,valid_point):
    res=np.zeros((1,100))
    for num in all_num:
        res=np.concatenate((res,np.expand_dims(num.flatten(),0)),axis=0)
    res=res[1:]
    for i in range(0,res.shape[1]):
        if (i//10,i%10) in valid_point:
            data[f'({i//10},{(i)%10})']=res[:,i]
    return data
    
def get_slice(temp,point_to_image_slice):
    return  point_to_image_slice[temp]
    
new_data=split_np_matrix(all_num,data,valid_point)#分离np_matrix数组，实际上可用explode进行处理
new_data.to_csv('new_data.csv')#这里用excel preview看数据就出现了19.0，但实际上通过调用columns属性就可以得出其实数据属性列是正常的


In [5]:
def generate_csv(data):
    data=pd.read_csv(data)
    column_name=data.columns.values.tolist()
    data=data.drop(['Unnamed: 0','np_mat_num','pid','source','pos','num','pd_prob','np_mat_num','dates'],axis=1)
    col=data.columns.values.tolist()
    index=[i%len(data) for i in range(0,len(data)*54)]
    data=pd.melt(data,id_vars=col[0:3],value_vars=col[3:],var_name='position',value_name='value')
    data=data.set_index(pd.Index(index)).sort_index()
    #res.to_csv('one_input_data')
    data['A'],data['B'],data['C']=data['image_path'].str.split('.',2).str
    data['slice']=data['position'].apply(get_slice,args=(point_to_image_slice,))
    data['slice']=data['slice'].map(lambda x:str(x))
    data['image_path']=data['A'].str.cat(data['B'],sep='.').str.cat(data['slice'],sep='_').str.cat(data['C'],sep='.')
    data.drop(['A','B','C'],axis=1,inplace=True)
    data.to_csv('one_input_data.csv')
    

In [6]:
generate_csv('new_data.csv')

/home/octusr3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'get_slice' is not defined

In [6]:
df=pd.read_csv('data.csv')#用来生成outpu下的文件夹,生成训练任务
id_base = 'pid'
date = '1005'
label_type = 'num'

for model, size, pos in product(['r50'], ['380'], ['disc', 'macula']):
    exp_name = f'{date}/{pos}_{label_type}_{model}_{size}'
    df_tmp = df.copy()
    df_tmp = df_tmp[df_tmp['pos'] == pos]
    df_tmp['dataset'] = df_tmp[id_base].map(to_dataset_mapping(df_tmp[id_base].tolist(), 5))
    ensure_path(f'output/{exp_name}/exp-{id_base}/tasks')
    df_tmp.to_csv(f'output/{exp_name}/exp-{id_base}/tasks/data.csv', index=False)

以下为数据后处理部分，用以生成原来形式的数据

In [2]:
def back_treat(res):
    res=pd.read_csv('res.csv')
    res['A'],res['B'],res['C']=res['image_path'].str.split('.',2).str
    res['image_path']=res['A'].str.cat(res['B'].str[0:3],sep='.').str.cat(res['C'],sep='.')
    res.drop(['A','B','C'],axis=1,inplace=True)
    data=pd.DataFrame(res.set_index(['image_path','position'])['pred'])
    df1=data.pivot_table(index='image_path',values='pred',columns='position')
    return df1


In [3]:
res=pd.read_csv('res.csv')
res['A'],res['B'],res['C']=res['image_path'].str.split('.',2).str
res['image_path']=res['A'].str.cat(res['B'].str[0:3],sep='.').str.cat(res['C'],sep='.')
res.drop(['A','B','C'],axis=1,inplace=True)
data=pd.DataFrame(res.set_index(['image_path','position'])['pred'])
df1=data.pivot_table(index='image_path',values='pred',columns='position')
original_data=df1.copy().reset_index()


/home/octusr3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


In [19]:
def concat_num(res,orignal_data):#用来将54个数字拼成数组
    temp=[]
    for i in res.values:
        temp.append(list(i)) 
    original_data['result']=temp
    return original_data

def convert_num_pred_to_str(tmp):
    cos=np.array(tmp).reshape(9,6)
    tep=[str(i).replace(' ','    ').replace('[','').replace(']','') for i in cos]
    re=''.join(x+' ' for x in tep)#不能 加\n,因为刚刚打印显示出来的字符串显示有'\n',并不是字符串里有的，而是就那么显示的
    return re
    #这里加' '是为了区分两个数字，不至于连到一起

original_data=concat_num(df1,original_data)
original_data=pd.DataFrame(original_data,columns=['image_path','result'])
original_data['pred']=original_data['result'].apply(convert_num_pred_to_str)

In [53]:
ripe_data=pd.read_csv('data.csv')
temp=ripe_data.drop_duplicates(subset=['image_path'])
temp=temp.drop(['pred'],axis=1)
res=temp.merge(original_data,how='right',on='image_path')
res.drop(['result'],axis=1,inplace=True)
res.to_csv('bupt.csv',index=False)#生成最后结果的代码


In [55]:
coret=pd.read_csv('bupt.csv')
coret

,image_path,source,pid,dates,eye,pos,num,pd_prob,dataset,pred
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143...,CASIA数据汇总-new,CRC-009-143,2018-06-26,OS,disc,[[ 0 0 0 0 0 0 0 0 0 0]\n [ 0 0 0 2...,[[0 0 0 0 0 0 0 0 0 0]\n [0 0 0 1 1 1 1 0 0 0]...,3,24.509548 27.689316 18.356636 22.3900...
1,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC000169/2...,CASIA数据汇总-new,CRC000169,2019-12-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 2...,1,25.561035 23.761473 25.265863 29.7103...
2,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-194/2019...,CASIA数据汇总-new,CS-194,2019-06-12,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,3,26.28671 29.038187 29.163105 28.0...
3,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-194/2019...,CASIA数据汇总-new,CS-194,2019-06-12,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,3,27.236834 28.491135 30.064913 30.5876...
4,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-195/2019...,CASIA数据汇总-new,CS-195,2019-06-12,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,1,23.294153 26.633678 28.874899 28.8087...
...,...,...,...,...,...,...,...,...,...,...
10593,20210820-sysu-oct/MAGIC4/MAGIC2238/20210628/OS...,MAGIC4,MAGIC2238,2021-06-28,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,4,26.968523 24.115074 25.735323 27.6409...
10594,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4,26.310482 28.819983 28.932184 31.0895...
10595,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4,26.588007 27.658792 28.928173 28.8422...
10596,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,MAGIC4,MAGIC2242,2021-07-13,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 3...,4,27.153723 28.263975 29.003323 25.2342...


经过测试以及论证，包括修改代码，发现在原始数据data.csv文件中，对一列数据进行统计  fds是9321条，fda是 4071条  正常
目前现有的最后做出来的结果数据 缺损数据  fda是3033   fds 7595 共10598条，无论怎么都对不上,后来经过测试数据发现原本的data.csv文件中有完全一模一样的两条数据
原始数据没有drop掉重复项，在直接预测原始的54个点的时候没有问题，但是分开做54点的时候，使用pivot，同一个元素就会有键覆盖的问题，最后导致数据总体数目不一样

In [209]:
a=df1.reset_index()['image_path']#数据检测异常代码
bupt=pd.read_csv('data.csv')['image_path']
d={'col1':a,'col2':bupt}
com=pd.DataFrame(data=d)
com.to_csv('feaf.csv')

In [74]:
temp=[]
for i in res.values:
   temp.append(list(i)) 
res['result']=temp

In [135]:
temp

,image_path,source,pid,dates,eye,pos,num,pd_prob,dataset
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0
1,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0
2,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-201/2019...,CASIA数据汇总-new,CS-201,2019-06-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 3...,0
3,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-201/2019...,CASIA数据汇总-new,CS-201,2019-06-13,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0
4,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-204/2019...,CASIA数据汇总-new,CS-204,2019-06-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0
...,...,...,...,...,...,...,...,...,...
13387,20210820-sysu-oct/MAGIC4/MAGIC2238/20210628/OS...,MAGIC4,MAGIC2238,2021-06-28,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,4
13388,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4
13389,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4
13390,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,MAGIC4,MAGIC2242,2021-07-13,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 3...,4
